# Doublet-Source Panel Method

In [ ]:
using Revise

include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")

using .PanelMethods: DoubletPanel2D, Uniform2D, grid_data, aero_coefficients, pressure_coefficient, ×
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST, linspace
using BenchmarkTools
using Plots
plotly()
# using PyPlot

In [ ]:
alpha_u = [0.1, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.1, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
airfoil = kulfan_CST(alphas, dzs, 0.2, 80);
plot(airfoil[:,1], airfoil[:,2], aspect_ratio=:equal)

### Setup

In [ ]:
panels = reverse([ DoubletPanel2D((xs, ys), (xe, ye)) for (xs, ys, xe, ye) ∈ (collect ∘ eachrow)([airfoil[2:end,:] airfoil[1:end-1,:]]) ], dims = 1)
uniform = Uniform2D(5.0, 5.0);

In [ ]:
@time (cl, kcl, cps) = aero_coefficients(panels, uniform)
println("Lift Coefficient: $cl, $kcl")

In [5]:
# Grid parameters
x_domain, y_domain = (-1, 2), (-1, 1)
grid_size = 50
x_dom, y_dom = linspace(x_domain..., grid_size), linspace(y_domain..., grid_size)
grid = x_dom × y_dom
# For plotting purposes
vels, pots = grid_data(panels, grid);

# Pressure coefficients
cp = pressure_coefficient.(vels, uniform.mag)
# print([ panel.source_strength for panel in panels ])

51×51 Array{Float64,2}:
 0.997945  0.997726  0.997485  0.997223  …  0.994012  0.994184  0.994407
 0.997807  0.997563  0.997293  0.996996     0.993278  0.993476  0.993735
 0.99766   0.997387  0.997084  0.996748     0.992416  0.992646  0.99295
 0.997503  0.997199  0.996858  0.996477     0.991399  0.991667  0.992028
 0.997336  0.996997  0.996613  0.99618      0.990188  0.990504  0.990936
 0.997159  0.996781  0.996349  0.995857  …  0.988736  0.989113  0.989635
 0.996972  0.996551  0.996065  0.995506     0.98698   0.987434  0.988071
 0.996776  0.996307  0.99576   0.995126     0.984834  0.985387  0.986176
 0.99657   0.996049  0.995435  0.994715     0.982184  0.982868  0.983856
 0.996356  0.995778  0.99509   0.994274     0.978876  0.979732  0.980987
 0.996135  0.995495  0.994726  0.993803  …  0.974692  0.97578   0.977399
 0.995908  0.995202  0.994345  0.993304     0.969324  0.970729  0.972852
 0.995678  0.994901  0.99395   0.992779     0.962325  0.964174  0.96701
 ⋮                           

In [6]:
plot(airfoil[:,1], airfoil[:,2], markershape = :circle, aspect_ratio = :equal)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
p1 = contour(x_dom, y_dom, cp, fill = true)
plot(p1)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), fill = true, aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl